In [ ]:
import pandas as pd

# Load CSVs with proper dtypes
students = pd.read_csv('students.csv', dtype={'UniqueID': str})
preferences = pd.read_csv('preference.csv', dtype={'UniqueID': str, 'CollegeID': str, 'PrefNumber': int})
seats = pd.read_csv('seat.csv', dtype={'CollegeID': str})

def seat_column(caste):
    if caste == 'SC':
        return 'SC'
    elif caste == 'ST':
        return 'ST'
    elif caste == 'BC':
        return 'BC'
    elif caste == 'Minority':
        return 'Minority'
    elif caste == 'OC':
        return 'OC'
    elif caste == 'SC-CC':
        return 'SC-CC'
    else:
        return None

output = []

# ✅ Ask user how many students to process
n = int(input("Enter number of students to process: "))

# Take first n students
for idx, srow in students.head(n).iterrows():
    uid = str(srow['UniqueID']).strip()
    gender = srow['Gender']
    caste = srow['Caste']
    
    # Get all preferences for this student sorted by PrefNumber
    pset = preferences[preferences['UniqueID'] == uid].sort_values(by='PrefNumber')
    if pset.empty:
        continue
    
    scol = seat_column(caste)
    allocated_college = None
    allocated_pref = None
    
    # Iterate through preferences until seat is found
    for _, pref in pset.iterrows():
        collegeid = str(pref['CollegeID']).strip()
        prefnumber = int(pref['PrefNumber'])
        
        seat_row = seats[seats['CollegeID'] == collegeid]
        if not seat_row.empty and scol and scol in seat_row.columns:
            try:
                seats_available = int(seat_row.iloc[0][scol])
            except:
                seats_available = 0
            
            if seats_available > 0:
                # Allocate seat
                seats.loc[seat_row.index, scol] = seats_available - 1
                seats.loc[seat_row.index, "TOTAL No. of students admitted"] = int(seat_row.iloc[0]["TOTAL No. of students admitted"]) + 1
                allocated_college = collegeid
                allocated_pref = prefnumber
                break  # stop after successful allocation
    
    # Record result (only required columns)
    output.append({
        'UniqueID': uid,
        'CollegeID': allocated_college,
        'PrefNumber': allocated_pref,
        'Gender': gender,
        'Caste': caste
    })

# Save to CSV (IDs kept as text, no scientific notation)
out_df = pd.DataFrame(output)
out_df['UniqueID'] = out_df['UniqueID'].astype(str)
out_df['CollegeID'] = out_df['CollegeID'].astype(str)

out_df.to_csv("student_allocations.csv", index=False,
              columns=['UniqueID','CollegeID','PrefNumber','Gender','Caste'])
